In [1]:
from abc import ABC, abstractmethod
from collections import deque

In [2]:
# data for the problem of travelling Romania
romania_map_graph = dict(
    Arad = ['Zerind', 'Sibiu', 'Timisoara'],
    Bucharest = ["Urziceni", "Pitesti", "Giurgiu", "Fagaras"],
    Craiova=["Drobeta", "Rimnicu", "Pitesti"],
    Drobeta=["Mehadia"],
    Eforie=["Hirsova"],
    Fagaras=["Sibiu"],
    Hirsova=["Urziceni"],
    Iasi=["Vaslui", "Neamt"],
    Lugoj=["Timisoara", "Mehadia"],
    Oradea=["Zerind", "Sibiu"],
    Pitesti=["Rimnicu"],
    Rimnicu=["Sibiu"],
    Urziceni=["Vaslui"]
)
romania_map_graph

{'Arad': ['Zerind', 'Sibiu', 'Timisoara'],
 'Bucharest': ['Urziceni', 'Pitesti', 'Giurgiu', 'Fagaras'],
 'Craiova': ['Drobeta', 'Rimnicu', 'Pitesti'],
 'Drobeta': ['Mehadia'],
 'Eforie': ['Hirsova'],
 'Fagaras': ['Sibiu'],
 'Hirsova': ['Urziceni'],
 'Iasi': ['Vaslui', 'Neamt'],
 'Lugoj': ['Timisoara', 'Mehadia'],
 'Oradea': ['Zerind', 'Sibiu'],
 'Pitesti': ['Rimnicu'],
 'Rimnicu': ['Sibiu'],
 'Urziceni': ['Vaslui']}

In [3]:
class Problem(ABC):
    def __init__(self, initial, goal=None):
        self.initial = initial
        self.goal = goal
        
    @abstractmethod
    def actions(self, state):
        pass
    
    @abstractmethod
    def result(self, state, action):
        pass
    
    def goal_test(self, state):
        if isinstance(self.goal, list):
            return any(x is state for x in self.goal)
        else:
            return state == self.goal

In [4]:
class GraphProblem(Problem):
    def __init__(self, initial, goal, graph):
        super().__init__(initial, goal)
        self.graph = graph
        
    def actions(self, A):
        return list(self.graph.get(A))
    
    def result(self, state, action):
        return action

In [5]:
class Graph:
    def __init__(self, graph_dict=None, directed=True):
        self.graph_dict = graph_dict or {}
        self.directed = directed
        if not directed:
            # make the graph undirected
            self.make_undirected()
            
    def make_undirected(self):
        for parent in list(self.graph_dict.keys()):    # keys of the graph dict
            for child in self.graph_dict[parent]:      # each value of the value list
                
                # if the value is in the dict i.e., update the list of the existing parents
                if child in self.graph_dict:
                    # update the list if the parent is not in it.
                    if parent not in self.graph_dict[child]:
                        self.connect(child, parent)
                
                # if the child in not in the dict
                else:
                    self.connect(child, parent)
                
    def connect(self, A, B):
        self.graph_dict.setdefault(A, []).append(B)
        
    
    def get(self, a, b=None):
        links = self.graph_dict.setdefault(a, {})
        return links if b is None else links.get(b)
    
    
def UndirectedGraph(graph_dict=None):
    return Graph(graph_dict=graph_dict, directed=False)

In [6]:
class Node:
    def __init__(self, state, parent=None, action=None):
        self.state = state
        self.parent = parent
        self.action = action
    
    def __repr__(self):
        return f"<Node {self.state}>"
    
    def expand(self, problem):
        return [self.child_node(problem, action)
               for action in problem.actions(self.state)]
    
    def child_node(self, problem, action):
        next_state = problem.result(self.state, action)
        next_node = Node(next_state, self, action)
        return next_node
    
    def path(self):
        node, path_back = self, []
        while node:
            path_back.append(node)
            node = node.parent
        return path_back[::-1]

In [7]:
a = romania_map_graph.copy()

a = UndirectedGraph(a)
a.graph_dict

{'Arad': ['Zerind', 'Sibiu', 'Timisoara'],
 'Bucharest': ['Urziceni', 'Pitesti', 'Giurgiu', 'Fagaras'],
 'Craiova': ['Drobeta', 'Rimnicu', 'Pitesti'],
 'Drobeta': ['Mehadia', 'Craiova'],
 'Eforie': ['Hirsova'],
 'Fagaras': ['Sibiu', 'Bucharest'],
 'Hirsova': ['Urziceni', 'Eforie'],
 'Iasi': ['Vaslui', 'Neamt'],
 'Lugoj': ['Timisoara', 'Mehadia'],
 'Oradea': ['Zerind', 'Sibiu'],
 'Pitesti': ['Rimnicu', 'Bucharest', 'Craiova'],
 'Rimnicu': ['Sibiu', 'Craiova', 'Pitesti'],
 'Urziceni': ['Vaslui', 'Bucharest', 'Hirsova'],
 'Zerind': ['Arad', 'Oradea'],
 'Sibiu': ['Arad', 'Fagaras', 'Oradea', 'Rimnicu'],
 'Timisoara': ['Arad', 'Lugoj'],
 'Giurgiu': ['Bucharest'],
 'Mehadia': ['Drobeta', 'Lugoj'],
 'Vaslui': ['Iasi', 'Urziceni'],
 'Neamt': ['Iasi']}

In [57]:
def depth_first_graph_search(problem):
    
    node = Node(problem.initial)
    frontier = deque([node])
    explored = set()
    
    while frontier:
        print(frontier)
        node = frontier.popleft()
        
        if problem.goal_test(node.state):
            return node.path()
        
        explored.add(node.state)
        
        for child in node.expand(problem):
            if child.state not in explored and child not in frontier:
                frontier.appendleft(child)
        
                
    return None

In [58]:
graph = UndirectedGraph(dict(
    S=["A", "D"],
    A=["B", "D"],
    D=["E"],
    B=["C", "E"],
    E=["F"],
    F=["G"]
))

In [59]:
map_problem = GraphProblem('S', 'G', graph)

path = depth_first_graph_search(map_problem)

deque([<Node S>])
deque([<Node D>, <Node A>])
deque([<Node A>, <Node E>, <Node A>])
deque([<Node B>, <Node E>, <Node A>])
deque([<Node E>, <Node C>, <Node E>, <Node A>])
deque([<Node F>, <Node C>, <Node E>, <Node A>])
deque([<Node G>, <Node C>, <Node E>, <Node A>])


In [60]:
path

[<Node S>, <Node D>, <Node A>, <Node B>, <Node E>, <Node F>, <Node G>]

In [51]:
for p in path:
    print(f"state: {p.state} parent: {p.parent}")

state: S parent: None
state: D parent: <Node S>
state: A parent: <Node D>
state: B parent: <Node A>
state: E parent: <Node B>
state: F parent: <Node E>
state: G parent: <Node F>
